In [1]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
# we initiate the StreamingContext with 10 second batch interval. next we initiate our sqlcontext
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [3]:
# initiate streaming text from a TCP (socket) source:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

In [4]:
# lines of tweets with socket_stream window of size 60, or 60 seconds windows of time
lines = socket_stream.window(60)

In [5]:
# just a tuple to assign names

from collections import namedtuple

fields = ("hashtag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [6]:
# here we apply different operations on the tweets and save them to a temporary sql table

( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) 
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a dataframe
  .limit(10).registerTempTable("tweets") ) ) # Registers only top 10 hashtags to a table.

In [7]:
# import libraries to visualize the results

import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
%matplotlib inline

In [8]:
# start streaming and wait couple of minutes to get enought tweets
ssc.start()

In [9]:
count = 0
while count < 5:
    
    time.sleep(5)
    top_10_tags = sqlContext.sql( 'Select hashtag, count from tweets' )
    top_10_df = top_10_tags.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="hashtag", data=top_10_df)
    plt.show()
    count = count + 1
    print(count)

AnalysisException: 'Table or view not found: tweets; line 1 pos 27'

In [ ]:
ssc.stop()

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
